In [3]:
import pandas as pd
# calculate tables of crash uniques
benchmark = "cflow"
expids = 5
fc = "fcb"
benchmarks = {"cflow", "jq", "mp42aac"} # "exiv2",
fcs = {"fca", "fcb"}

In [4]:
for fc in fcs:
    dfs = []
    for expid in range(0,expids):
        df = pd.read_csv(f"csv/fscve_unique_crashes_{benchmark}_{fc}_{expid+1}.csv")
        dfs.append(df)
    df_unique_crashes_all = pd.concat(dfs, ignore_index=True)
    cols_to_keep = ["benchmark","fuzzer_combination","experiment_id","count"]
    df_unique_crashes_all.drop(df_unique_crashes_all.columns.difference(cols_to_keep), axis=1, inplace=True)
    print(df_unique_crashes_all)
    path_unique_crashes_all_csv = f"csv/fscve_unique_crashes_{benchmark}_{fc}_all.csv"
    df_unique_crashes_all.to_csv(path_unique_crashes_all_csv)


  benchmark fuzzer_combination  experiment_id  count
0     cflow                fca              1    743
1     cflow                fca              2    454
2     cflow                fca              3    671
3     cflow                fca              4    516
4     cflow                fca              5    485
  benchmark fuzzer_combination  experiment_id  count
0     cflow                fcb              1    326
1     cflow                fcb              2    112
2     cflow                fcb              3    577
3     cflow                fcb              4    493
4     cflow                fcb              5    237


In [5]:
#calc total unique crashes of a fuzzer combination on a benchmark
dfres = pd.DataFrame()
df = pd.DataFrame()
cols_to_keep = ["benchmark","fuzzer_combination","testcase_hash"]
df_results = []
#calc all data of a fuzzer combination on one benchmark
for fc in fcs:
    dfs = []
    for expid in range(0,expids):
        df = pd.read_csv(f"csv/fscve_unique_crashes_step7metric1_{benchmark}_{fc}_{expid+1}.csv")
        dfs.append(df)
    dfres = pd.concat(dfs, ignore_index=True)
    dfres.drop(dfres.columns.difference(cols_to_keep), axis=1, inplace=True)
    #print(dfres)
    # print(dfres.shape[1])
    dfres =  dfres.drop_duplicates(["testcase_hash"])
    # print(dfres.shape[1])
    dfres.to_csv(f"csv/fscve_unique_crashes_step7metric1_{benchmark}_{fc}.csv")
    # print(dfres)
    df_results.append(dfres)

#merge two fuzzer combination
df_result_benchmark = pd.concat(df_results, ignore_index=True)
df_result_drop = df_result_benchmark.drop(dfres.columns.difference(cols_to_keep), axis=1)
print("*"*30)
print(df_result_drop)
print("-"*30)


df_unique_crashes_total_count = df_result_drop.groupby(["benchmark", "fuzzer_combination"])[["testcase_hash"]].count().reset_index(drop=False)
# print(df_unique_crashes_total_count)
# print(df_unique_crashes_total_count["testcase_hash"])
#add column
df_result_final_crash_step7metric1=pd.DataFrame()
df_result_final_crash_step7metric1["benchmark"]=df_result_drop.drop_duplicates(["benchmark", "fuzzer_combination"])["benchmark"]
df_result_final_crash_step7metric1["fuzzer_combination"]=df_result_drop.drop_duplicates(["benchmark", "fuzzer_combination"])["fuzzer_combination"]
df_result_final_crash_step7metric1["total_unique"]=df_unique_crashes_total_count["testcase_hash"].to_list()
df_result_final_crash_step7metric1=df_result_final_crash_step7metric1.reset_index(drop=True)
df_result_final_crash_step7metric1.to_csv(f"csv/fscve_crash_step7metric1_totalunique_{benchmark}.csv")
# print(df_result_final_crash_step7metric1)


******************************
     benchmark fuzzer_combination  \
0        cflow                fca   
1        cflow                fca   
2        cflow                fca   
3        cflow                fca   
4        cflow                fca   
...        ...                ...   
4597     cflow                fcb   
4598     cflow                fcb   
4599     cflow                fcb   
4600     cflow                fcb   
4601     cflow                fcb   

                                          testcase_hash  
0     6efe29740e288d39419180ed180ad504301a5f812b4bb0...  
1     c0933a6d225bc6ffb4e2bcaad34d0b4581a768290f8dfb...  
2     9d8e83635e429cb0ca9edac85d61940ef53f1b4bdf2c06...  
3     5002388564e8e820387f613162dd25ffd831d40fb58364...  
4     b7a11deb43440f0efc8f48ab3699a90fc5d149c6d954d2...  
...                                                 ...  
4597  5d7938b1fc66b6f1dce54e9609e2a6e0fc3a3d6da1e142...  
4598  a381eb018f15c6cc35f2958033782563fba421bba33979...  
45

In [6]:
#calc for unique real bugs
for fc in fcs:
    dfs = []
    for expid in range(0,expids):
        df = pd.read_csv(f"csv/fscve_crash_analysis_bugcnt_{benchmark}_{fc}_{expid+1}.csv")
        dfs.append(df)
    df_bugcnt_all = pd.concat(dfs, ignore_index=True)
    cols_to_keep = ["benchmark","fuzzer_combination","experiment_id","bug_count"]
    df_bugcnt_all.drop(df_bugcnt_all.columns.difference(cols_to_keep), axis=1, inplace=True)
    print(df_bugcnt_all)
    path_bugcnt_all_csv = f"csv/fscve_crash_analysis_bugcnt_{benchmark}_{fc}_all.csv"
    df_bugcnt_all.to_csv(path_bugcnt_all_csv)

  benchmark fuzzer_combination  experiment_id  bug_count
0     cflow                fca              1          5
1     cflow                fca              2          5
2     cflow                fca              3          3
3     cflow                fca              4          4
4     cflow                fca              5          4
  benchmark fuzzer_combination  experiment_id  bug_count
0     cflow                fcb              1          3
1     cflow                fcb              2          4
2     cflow                fcb              3          3
3     cflow                fcb              4          2
4     cflow                fcb              5          4


In [7]:
#aggregate the bugcount data of repeated experiments ,the bugs are surely not unique ,only to represent the ability of a fuzzer combination on a benchmark during all the repeated times
df = pd.DataFrame()
dfres = pd.DataFrame()
cols_to_keep = ["benchmark","fuzzer_combination"]
df_results = []
#calc all data of a fuzzer combination on one benchmark
for fc in fcs:
    for expid in range(0,expids):
        df = pd.read_csv(f"csv/fscve_crash_analysis_bugcnt_{benchmark}_{fc}_{expid+1}.csv")
        df[f"experiment_id_{expid + 1}"] = df.loc[0,"bug_count"]
        cols_to_keep.append(f"experiment_id_{expid + 1}")
        # print(df)
        dfres.drop(dfres.columns.difference(cols_to_keep), axis=1, inplace=True)
        if expid > 0:
            dfres = pd.merge(dfres, df, on=["benchmark","fuzzer_combination"])
        else :
            dfres = df
    dfres.drop(dfres.columns.difference(cols_to_keep), axis=1, inplace=True)
    dfres.to_csv(f"csv/fscve_crash_analysis_bugcnt_{benchmark}_{fc}.csv")
    # print(dfres)
    df_results.append(dfres)

#merge two fuzzer combination
df_result_final_bugcnt = pd.concat(df_results)
cols_to_keep.remove("benchmark")
cols_to_keep.remove("fuzzer_combination")
print(cols_to_keep)
print("*"*20)
print()
#dron non-digit columns for calculating
df_result_drop = df_result_final_bugcnt.drop(dfres.columns.difference(cols_to_keep), axis=1)
# print(df_result_final)
# print("*"*30)
# print(df_result_drop)
# print("-"*30)
df_result_final_bugcnt["max"] = df_result_drop.max(axis=1)
df_result_final_bugcnt["avg"] = df_result_drop.mean(axis=1)
df_result_final_bugcnt["avg"] = df_result_final_bugcnt["avg"].map(lambda x:int(x))  #float to int
print(df_result_final_bugcnt)
df_result_final_bugcnt.to_csv(f"csv/fscve_crash_analysis_bugcnt_repeated_{benchmark}.csv")

['experiment_id_1', 'experiment_id_2', 'experiment_id_3', 'experiment_id_4', 'experiment_id_5', 'experiment_id_1', 'experiment_id_2', 'experiment_id_3', 'experiment_id_4', 'experiment_id_5']
********************

  benchmark fuzzer_combination  experiment_id_1  experiment_id_2  \
0     cflow                fca                5                5   
0     cflow                fcb                3                4   

   experiment_id_3  experiment_id_4  experiment_id_5  max  avg  
0                3                4                4    5    4  
0                3                2                4    4    3  


In [8]:
#calc for unique real bug ,the time of bug find first , and calc the total number of unique real bugs of a fuzzer combination on one benchmark during all the repeated times
df = pd.DataFrame()
dfres = pd.DataFrame()
cols_to_keep = ["discovery_time","crash_frames_hash"]
df_results = []
#calc all data of a fuzzer combination on one benchmark
dict_unique_bugcnt = [] #unique bug count of one fuzzer combination
for fc in fcs:
    dfs = []
    for expid in range(0,expids):
        df = pd.read_csv(f"csv/fscve_crash_analysis_bugfindfirsttime_{benchmark}_{fc}_{expid+1}.csv")
        df.drop(df.columns.difference(cols_to_keep), axis=1, inplace=True)
        dfs.append(df)
    dfres=pd.concat(dfs, ignore_index=True)
    dfres = dfres.drop_duplicates(['crash_frames_hash']) #record once for special fuzzer and real bug
    dfres.to_csv(f"csv/fscve_crash_analysis_bugfindfirsttime_{benchmark}_{fc}.csv")
    dict_unique_bugcnt.append(dfres.shape[0])

    dfres["fuzzer_combination"] = [f"{fc}" for i in range(0,dfres.shape[0])]
    print(dfres)
    df_results.append(dfres)

# calc the total number of unique real bugs of a fuzzer combination on one benchmark during all the repeated times
df_result_final_bugcnt["unique_bugcnt"] = dict_unique_bugcnt
#print(df_result_final_bugcnt)
df_result_final_bugcnt.to_csv(f"csv/fscve_crash_analysis_bugcnt_unique_{benchmark}.csv")


#merge two fuzzer combination to decide which fuzzer combination finds the same special bug faster
df_result_final_bugfindfirsttime = pd.concat(df_results)

#keep the same bugs --> sort by bug and time --->  drop duplicate ---> sort by time
df_result_final_bugfindfirsttime = df_result_final_bugfindfirsttime[df_result_final_bugfindfirsttime.duplicated('crash_frames_hash', keep=False)].sort_values(['crash_frames_hash',"discovery_time"]).drop_duplicates(['crash_frames_hash']).sort_values(["discovery_time"]).reset_index(drop=True)
print(df_result_final_bugfindfirsttime)
df_result_final_bugfindfirsttime.to_csv(f"csv/fscve_crash_analysis_bugdetail_findfirst_{benchmark}.csv")

#add column
df_result_final_bugfindfirsttime["bugcnt_findfirst"]=df_result_final_bugfindfirsttime["fuzzer_combination"]
df_result_final_bugfindfirsttime = df_result_final_bugfindfirsttime.groupby("fuzzer_combination")[["bugcnt_findfirst"]].count().reset_index(drop=False)

df_result_final_bugfindfirsttime.to_csv(f"csv/fscve_crash_analysis_bugcnt_findfirst_{benchmark}.csv")
print(df_result_final_bugfindfirsttime)




#for the same special bug,which fuzzer  combination finds
# #dron non-digit columns for calculating
# df_result_drop = df_result_final.drop(dfres.columns.difference(cols_to_keep), axis=1)
# # print(df_result_final)
# # print("*"*30)
# # print(df_result_drop)
# # print("-"*30)
# df_result_final["max"] = df_result_drop.max(axis=1)
# df_result_final["avg"] = df_result_drop.sum(axis=1)//expids
# print(df_result_final)
# df_result_final.to_csv(f"csv/fscve_crash_analysis_bugcnt_final_{benchmark}.csv")

    discovery_time                 crash_frames_hash fuzzer_combination
0                5  9c74ce5d00fb534a6644db103f1bfe0b                fca
1               33  99fad7a3b490f7d3d7fc13b4db848e11                fca
2              122  bf39470c4385d389d7814248ff4d17c1                fca
3              626  333b4b08bdbba43f4ade91fe58aa064e                fca
4             1289  097576036382d7d2b774623b1844285b                fca
6              192  0385fd953418b4a9a73109965ed568c1                fca
15             861  c35e68ad11ee23588788f0dd3a2c0d73                fca
    discovery_time                 crash_frames_hash fuzzer_combination
0               10  9c74ce5d00fb534a6644db103f1bfe0b                fcb
1              231  bf39470c4385d389d7814248ff4d17c1                fcb
2              501  5cb48614635b636e96efbb73c8add63f                fcb
5              401  d05f359b27234cc1572a4c4af1b79e43                fcb
6             1014  99fad7a3b490f7d3d7fc13b4db848e11            